In [1]:
%pylab inline
import tensorflow as tf
import struct, gzip
import Image
from tensorflow.examples.tutorials.mnist import input_data

Populating the interactive namespace from numpy and matplotlib


In [2]:
data = genfromtxt('../data/exem/machine-error01.csv', skip_header=True, delimiter=',')
errors = data[:, 0]
data = data[:,2:]

train_errors = errors[:11000]
train_data = data[:11000, :]

test_errors = errors[11000:]
test_data = data[11000:, :]

In [3]:
# Parameters
learning_rate = 0.001
training_epochs = 20
batch_size = 100
display_step = 1
n_steps = 6 * 60

# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 30 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input], name='x')
y = tf.placeholder("float", [None, 2], name='y')

In [4]:
def next_batch(data, valid_data, n_sample=360, n_steps=n_steps):
    nrow, ncol = data.shape
    maximum = nrow - n_sample - n_steps
    idx = randint(0, maximum)
    batch_xs = data[idx:idx+n_sample,:]
    batch_y = valid_data[idx + n_sample + n_steps:idx + n_sample + n_steps + 1]
    return batch_xs, batch_y

while True:
    batch_xs, batch_y = next_batch(train_data, train_errors)
    print batch_xs, batch_y
    if batch_y == 1.:
        break


[[  41.  344.   27. ...,   48.  461.   78.]
 [  41.  344.   27. ...,   48.  461.   78.]
 [  41.  344.   27. ...,   48.  461.   78.]
 ..., 
 [  41.   33.   23. ...,   48.  461.   78.]
 [  41.   33.   25. ...,   48.  461.   78.]
 [  41.   33.   23. ...,   48.  461.   78.]] [ 1.]


In [10]:
weight_1 = tf.Variable(tf.random_normal([n_input, n_hidden_1]), name='weight1')
bias_1 = tf.Variable(tf.random_normal([n_hidden_1]), name='bias1')
layer_1 = tf.add(tf.matmul(x, weight_1), bias_1, name='layer1')
layer_1 = tf.nn.relu(layer_1)

weight_2 = tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]), name='weight2')
bias_2 = tf.Variable(tf.random_normal([n_hidden_2]), name='bias2')
layer_2 = tf.add(tf.matmul(layer_1, weight_2), bias_2, name='layer2')
layer_2 = tf.nn.relu(layer_2)

weight_3 = tf.Variable(tf.random_normal([n_hidden_2, 1]))
bias_3 = tf.Variable(tf.random_normal([1]))
layer_3 = tf.add(tf.matmul(layer_2, weight_3), bias_3)
layer_3 = tf.nn.relu(layer_3)

layer_4 = tf.sigmoid(tf.reduce_sum(layer_3))

y_ = tf.placeholder(tf.float32, [None, 2])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[0]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)


print x.get_shape()
print weight_1.get_shape()
print layer_1.get_shape()
print layer_3.get_shape()
print layer_4.get_shape()
# print cross_entropy

ValueError: No gradients provided for any variable: ((None, <tensorflow.python.ops.variables.Variable object at 0x7f3f0490e7d0>), (None, <tensorflow.python.ops.variables.Variable object at 0x7f3f044a0e90>), (None, <tensorflow.python.ops.variables.Variable object at 0x7f3f0490ebd0>), (None, <tensorflow.python.ops.variables.Variable object at 0x7f3f044aca10>), (None, <tensorflow.python.ops.variables.Variable object at 0x7f3f0448b190>), (None, <tensorflow.python.ops.variables.Variable object at 0x7f3f04438fd0>), (None, <tensorflow.python.ops.variables.Variable object at 0x7f3f0490ed10>), (None, <tensorflow.python.ops.variables.Variable object at 0x7f3f0490e710>), (None, <tensorflow.python.ops.variables.Variable object at 0x7f3f042b7f90>), (None, <tensorflow.python.ops.variables.Variable object at 0x7f3f042b7ad0>), (None, <tensorflow.python.ops.variables.Variable object at 0x7f3f043589d0>), (None, <tensorflow.python.ops.variables.Variable object at 0x7f3f042c96d0>), (None, <tensorflow.pyth

In [ ]:
# Initializing the variables
init = tf.initialize_all_variables()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    batch_x, batch_y = next_batch(train_data, train_errors)
    print batch_x, batch_y
    result = sess.run([layer_4], feed_dict={x: batch_x, y: [batch_y]})
    print np.shape(result[0]), result[0]
    print sess.run([cross_entropy], feed_dict={x: batch_x, y: [batch_y], y_: result[0]})
#     print sess.run([cross_entropy], feed_dict={y_:batch_y})